In [1]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from skimage.feature import hog
from tqdm import tqdm

In [2]:
# Function to load and resize images from a folder
def load_and_resize_images_from_folder(folder, size=(128, 128)):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            resized_img = cv2.resize(img, size)  
            images.append(resized_img)
            filenames.append(filename)
    return images, filenames



In [7]:
# Function to extract HOG features from images
def extract_hog_features(images):
    features = []
    for image in tqdm(images, desc="Extracting HOG features"):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        
        hog_feature = hog(
            gray, 
            orientations=9, 
            pixels_per_cell=(8, 8), 
            cells_per_block=(2, 2), 
            block_norm='L2-Hys', 
            visualize=False, 
            transform_sqrt=True
        )
        features.append(hog_feature)
    return np.array(features)  



In [4]:
# Function to cluster images and save them in output folders
def cluster_and_save_images(features, filenames, images, output_dir, n_clusters=4):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(features)

    for i in range(n_clusters):
        cluster_dir = os.path.join(output_dir, f'player{i}')
        os.makedirs(cluster_dir, exist_ok=True)

    for label, filename, img in zip(labels, filenames, images):
        cluster_dir = os.path.join(output_dir, f'player{label}')
        output_path = os.path.join(cluster_dir, filename)
        cv2.imwrite(output_path, img)

In [ ]:
def main():
    top_dir = 'DataSet/two_players_top'
    bot_dir = 'DataSet/two_players_bot'
    output_dir = 'output'

    print("Loading and resizing images...")
    top_images, top_filenames = load_and_resize_images_from_folder(top_dir)
    bot_images, bot_filenames = load_and_resize_images_from_folder(bot_dir)

    images = top_images + bot_images
    filenames = top_filenames + bot_filenames

    print("Extracting HOG features...")
    features = extract_hog_features(images)

    print("Clustering images...")
    cluster_and_save_images(features, filenames, images, output_dir, n_clusters=4)

    print(f"Images clustered into {output_dir}/player0, player1, player2, player3")

if __name__ == "__main__":
    main()